In [0]:
%run  ../connector/sql_connector

In [0]:
from pyspark.sql.functions import col

dim_fornecedor_df = spark.table("""dev.bronze.dim_fornecedores""").drop("id").select("fornecedor_id", "nome_fornecedor")
dim_produtos_df = spark.table("""dev.bronze.dim_produtos""").drop("id")
dim_usuarios_df = spark.table("""dev.bronze.dim_usuarios""").drop("id")
fato_farmacia_df = spark.table("""dev.bronze.fato_farmacia""").drop("id")

In [0]:
cols_select = ["venda_id", "nome_usuario", "cidade", "estado", "pais",
    "quantidade", "valor_total", "to_date(data_venda) as data_venda", "nome_fornecedor",
    "nome_produto", "categoria"]

fato_farmacia_df \
    .join(dim_produtos_df, ["produto_id"]) \
    .join(dim_fornecedor_df, ["fornecedor_id"]) \
    .join(dim_usuarios_df, ["usuario_id"]) \
    .filter(~col("marca").isin(["Generics"])) \
    .selectExpr(*cols_select) \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("dev.silver.stage__farmacia")